# HW1. Полина Машковцева

## 1. Сбор корпуса

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


датасет - **500N-KPCrowd-v1.1** (из статьи https://arxiv.org/abs/1306.4606). описание оттуда: 500N-KPCrowd-v1.1 is a broadcast news transcription dataset. This dataset consists of 500 English broadcast news stories from 10 different categories (art and culture; business; crime; fashion; health; politics us; politics world; science; sports; technology) with 50 docs per category. The ground truth is built using Amazon’s Mechanical Turk service to recruit and manage taggers. Multiple annotators were required to look at the same news story and assign a set of keywords from the text itself. The final ground truth consists of keywords selected at least by 90% of the taggers.

In [ ]:
! wget https://github.com/LIAAD/KeywordExtractor-Datasets/raw/master/datasets/500N-KPCrowd-v1.1.zip -P drive/My\ Drive

In [ ]:
! unzip 500N-KPCrowd-v1.1.zip -d drive/My\ Drive/data

в архиве содержатся две папки: docsutf8 и keys. в docsutf8 находятся тексты статей в формате *.txt*, в keys - ключевые слова в формате *.key*. название файла с текстом статьи соотносится с названием файла с ключевыми словами.

если посмотреть тексты статей, мы увидим, что ~50 из них - просто новостные заголовки (видимо, какая-то ошибка или лаг парсинга при составлении датасета; с ключевыми словами к таким файлам все в порядке). нам такое не нужно, поэтому при сборе корпуса будем отфильтровывать тексты, в которых содержится меньше 21 слова (фан факт: средняя длина заголовка в англоязычных новостных изданиях - 120 символов; примерно 26 слов при средней длине 4.7 слова в английском языке).

In [3]:
from pathlib import Path
import os
from os import listdir

In [4]:
articles_directory = 'drive/My Drive/data/500N-KPCrowd-v1.1/docsutf8/'
keywords_directory = 'drive/My Drive/data/500N-KPCrowd-v1.1/keys/'

In [5]:
#собираем корпус-словарь (длиной ~5000 символов) из текстов статей и соответствующих им ключевых слов

articles = {}
corpus_length = 0

for article in listdir(articles_directory):
  articles_file = os.path.join(articles_directory, article)
  with open(articles_file, 'r', encoding='utf-8') as a:
    text = a.read()
  keywords_file = os.path.join(keywords_directory, article[:-4] + '.key')
  with open(keywords_file, 'r', encoding='utf-8') as k:
    words = k.read()
  if len(text.split()) > 20:
    articles.update({text: words.split('\n')})
  corpus_length += len(text.split())
  if corpus_length >= 5000:
    break

## 2. Ручная разметка

прочитаем загруженные тексты (буквально).

In [8]:
len(articles.keys())

15

In [7]:
for key in articles.keys():
  print(key + '\n\n')

Broncos top brass skip the workout of D-lineman Bowers
Clemson defensive end Da'Quan Bowers threw a workout party Friday, and the Broncos' top football decision-makers did not attend. Bowers, whose right knee surgery earlier this year has threatened his top-10 status in the NFL draft, worked out at Clemson University while Broncos executive vice president of football operations John Elway, general manager Brian Xanders and coach John Fox elected to stay home and meet with Washington quarterback Jake Locker instead. Locker was at the team's Dove Valley complex. The Broncos, who have repeatedly said they like their quarterback triumvirate of Kyle Orton, Tim Tebow and Brady Quinn, also attended Missouri quarterback Blaine Gabbert's workout last month and are scheduled to be the last stop on Auburn quarterback Cam Newton's tour of teams later this month. Locker had been the top-ranked senior prospect by many teams before the 2010 season began, but he saw his draft stock take a hit as the H

вручную подберем к ним ключевые слова.

In [9]:
keywords_manual = [['Clemson', 'Da\'Quan Bowers', 'football', 'Broncos', 'NFL', 'Dove Valley', 'Huskies', 'Holiday Bowl'], ['genetic tests', 'genetics', 'health', 'diseases', 'U.S. Food and Drug Administration', 'Sarah Lawrence College', 'genetic counseling', 'DNA'], ['Barack Obama', 'Obama', 'Austria', 'robbery', 'bank robbery'], ['federal budget negotiations', 'Tea Party', 'Capitol', 'Republicans', 'government shutdown'], ['coal mine accident', 'China', 'Xinjiang', 'coal mine', 'coal outburst', 'gas outburst', 'Urumqi'], ['Sunday news TV shows', 'Washington', 'This Week', 'Meet the Press', 'Face the Nation', 'State of the Union'], ['Sony', 'iPhone 5', 'Wall Street Journal', 'Japan', 'earthquake', 'tsunami', 'image sensors', 'Apple', 'OmniVision'], ['calories', 'calorie counting', 'dieting', 'Food and Drug Administration', 'FDA', 'health overhaul legislation', 'eating out', 'obesity', 'Center for Science in the Public Interest', 'nutrition', 'health'], ['Iraq', 'air pollution', 'University of Alaska', 'Anaheim', 'CA', 'California', 'American Chemical Society', 'Iraq War', 'USA', 'war'], ['cold', 'allergy', 'ACAAI', 'immune system', 'disease symptoms', 'allergic rhinitis', 'hay fever'], ['federal court', 'NH', 'New Hampshire', 'health care', 'health insurance', 'Medicare'], ['shopping', 'Good Morning America', 'GMA', 'sale', 'discounts', 'Where the Weekend Sales Are', 'weekend sales', 'fashion', 'clothing', 'jewelry', 'shoes', 'toys'], ['Dita Von Teese', 'burlesque', 'L.A.', 'fashion', 'Louis Marie De Castelbajac', 'models', 'Griffith Observatory', 'dancers', 'Vivienne Westwood'], ['calorie counts', 'Food and Drug Administration', 'FDA', 'obesity', 'calorie consumption', 'eating out', 'nutrition', 'Center for Science in the Public Interest', 'National Restaurant Association', 'health'], ['unemployment', 'unemployment drop', 'stocks', 'investment']]

посчитаем сходство. на всякий случай будем переводить все элементы в нижний регистр.

In [62]:
def evaluate_keyword_extraction(golden: list, target: list):
  precisions = []
  recalls = []
  f1_scores = []
  for i in range(len(golden)):
    golden_lower = list(map(lambda j: j.lower(), list(golden)[i]))
    target_lower = list(map(lambda k: k.lower(),list(target)[i]))
    intersection = len(set(golden_lower) & set(target_lower))
    precision = intersection / len(golden_lower)
    precisions.append(precision)
    recall = intersection / len(target_lower)
    recalls.append(recall)
    # print(f'Precision for text {i+1} is {round((precision), 3)}')
    if (precision + recall) != 0:
      f1_score = 2 * (precision * recall) / (precision + recall)
      f1_scores.append(f1_score)
      # print(f'Recall for text {i+1} is {round((recall), 3)}')
      # print(f'F1-score for text {i+1} is {round((f1_score), 3)}\n\n')
    # else:
    #   print(f'Recall for text {i+1} is {round((recall), 3)}\n\n')
  average_precision = sum(precisions) / len(precisions)
  average_recall = sum(recalls) / len(recalls)
  average_f1_score = sum(f1_scores) / len(f1_scores)
  return average_precision, average_recall, average_f1_score

In [39]:
manual_average_precision, manual_average_recall, manual_average_f1_score = evaluate_keyword_extraction(articles.values(), keywords_manual)

In [40]:
print(f'Average precision is {manual_average_precision}')
print(f'Average recall is {manual_average_recall}')
print(f'Average f1_score is {manual_average_f1_score}')

Average precision is 0.06505710545225313
Average recall is 0.3677873977873978
Average f1_score is 0.11582914662604668


ну такое... сделаем эталоном разметки пересечение корпусной и ручной.

In [13]:
golden_standard = []

for i in range(len(articles.values())):
  corpus_lower = list(map(lambda j: j.lower(), list(articles.values())[i]))
  manual_lower = list(map(lambda k: k.lower(), keywords_manual[i]))
  golden_standard.append(list(corpus_lower + manual_lower))

## 3. Извлечение ключевых слов

будем брать топ-15 ключевых слов/фраз (чтобы не выходило слишком много).

### 3.1. TextRank

In [ ]:
! python3 -m pip install pytextrank

In [ ]:
! python3 -m spacy download en_core_web_sm

In [16]:
import spacy
import pytextrank

In [17]:
nlp = spacy.load('en_core_web_sm')

In [18]:
nlp.add_pipe('textrank')

In [19]:
#собираем корпус-словарь из текстов статей и соответствующих им ключевых слов/фраз по textrank

articles_textrank = {}

for text in articles.keys():
  keywords_textrank = []
  doc = nlp(text)
  for phrase in doc._.phrases:
    keywords_textrank.append(phrase.text)
  if len(keywords_textrank) > 15:
    articles_textrank.update({text: keywords_textrank[:15]})
  else:
    articles_textrank.update({text: keywords_textrank})

### 3.2. keyBERT

In [ ]:
! pip install keybert

In [21]:
from keybert import KeyBERT

In [ ]:
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')

с keyBERT есть проблема: он показывает нам ключевые н-граммы только в заданных рамках. для +-(так себе)сбалансированности попросим его выдать нам 7 униграмм и 8 биграмм:

In [23]:
#собираем корпус-словарь из текстов статей и соответствующих им ключевых слов/фраз по keybert

articles_keybert = {}

for text in articles.keys():
    keywords_keybert = []
    keyword_unigrams = kw_extractor.extract_keywords(text, keyphrase_ngram_range=(1, 1), top_n=7)
    for item in keyword_unigrams:
      keywords_keybert.append(item[0])
    keyword_bigrams = kw_extractor.extract_keywords(text, keyphrase_ngram_range=(1, 2), top_n=8)
    for items in keyword_bigrams:
      keywords_keybert.append(items[0])
    articles_keybert.update({text: keywords_keybert})

### 3.3. Gensim

In [ ]:
! pip install gensim

In [41]:
from gensim.summarization import keywords

In [42]:
#собираем корпус-словарь из текстов статей и соответствующих им ключевых слов/фраз по gensim

articles_gensim = {}

for text in articles.keys():
  if len(keywords(text)) > 15:
    articles_gensim.update({text: keywords(text).split('\n')[:15]})
  else:
    articles_gensim.update({text: keywords(text).split('\n')})

## 4. Шаблоны

посмотрим, какие pos-сочетания чаще всего являются ключевыми словами (помня о несбалансированности выборки из-за keyBERT). интуитивно (для английского языка) это NOUN, NOUN+NOUN и ADJ+NOUN.

In [43]:
#собираем список всех ключевых слов

keywords_all = []

for keywords_set in articles_textrank.values(), articles_keybert.values(), articles_gensim.values():
  for words in keywords_set:
    for keyphrase in words:
      keywords_all.append(keyphrase)

In [44]:
#собираем словарь ключевых слов/фраз и соответствующих им pos-тегов

keywords_pos = {}

for keyphrase in keywords_all:
  if len(keyphrase.split()) > 1:
    phrase_pos = []
    doc = nlp(keyphrase)
    for token in doc:
      phrase_pos.append(token.pos_)
    keywords_pos.update({keyphrase: '+'.join(phrase_pos)})
  else:
    doc = nlp(keyphrase)
    for token in doc:
       keywords_pos.update({keyphrase: token.pos_})

In [45]:
from collections import Counter

возьмем топ-10.

In [46]:
top = Counter(list(keywords_pos.values())).most_common(10)
top

[('NOUN', 154),
 ('PROPN', 62),
 ('NOUN+NOUN', 56),
 ('ADJ+NOUN', 44),
 ('VERB', 42),
 ('PROPN+PROPN', 33),
 ('PROPN+NOUN', 22),
 ('ADJ', 18),
 ('VERB+NOUN', 16),
 ('NUM+NOUN', 12)]

In [47]:
top_list = []
for i in range(len(top)):
  top_list.append(top[i][0])

In [48]:
#фильтруем ключевые слова из корпуса

def filter_keywords(articles_dictionary: list) -> list:
  articles_filtered = {}
  for k, v in articles_dictionary.items():
    filtered_keyphrases = []
    for item in v:
      for key, value in keywords_pos.items():
        if item == key and value in top_list:
          filtered_keyphrases.append(item)
        articles_filtered.update({k: filtered_keyphrases})
  return articles_filtered

In [49]:
filtered_textrank = filter_keywords(articles_textrank)
filtered_keybert = filter_keywords(articles_keybert)
filtered_gensim = filter_keywords(articles_gensim)

## Оценка методов

### 5.1. Без фильтра

In [50]:
precisions = {}
recalls = {}
f1_scores = {}

In [51]:
articles_textrank_precision, articles_textrank_recall, articles_textrank_f1_score = evaluate_keyword_extraction(golden_standard, list(articles_textrank.values()))
precisions.update({'textrank': articles_textrank_precision})
recalls.update({'textrank': articles_textrank_recall})
f1_scores.update({'textrank': articles_textrank_f1_score})

In [52]:
articles_keybert_precision, articles_keybert_recall, articles_keybert_f1_score = evaluate_keyword_extraction(golden_standard, list(articles_keybert.values()))
precisions.update({'keybert': articles_keybert_precision})
recalls.update({'keybert': articles_keybert_recall})
f1_scores.update({'keybert': articles_keybert_f1_score})

In [53]:
articles_gensim_precision, articles_gensim_recall, articles_gensim_f1_score = evaluate_keyword_extraction(golden_standard, list(articles_gensim.values()))
precisions.update({'gensim': articles_gensim_precision})
recalls.update({'gensim': articles_gensim_recall})
f1_scores.update({'gensim': articles_gensim_f1_score})

In [54]:
print(f'The best average precision is {max(precisions.values())} by {list(precisions.keys())[list(precisions.values()).index(max(precisions.values()))]} method')
print(f'The best average recall is {max(recalls.values())} by {list(recalls.keys())[list(recalls.values()).index(max(recalls.values()))]} method')
print(f'The best average f1-score is {max(f1_scores.values())} by {list(f1_scores.keys())[list(f1_scores.values()).index(max(f1_scores.values()))]} method')

The best average precision is 0.09661584261660632 by textrank method
The best average recall is 0.36642857142857144 by gensim method
The best average f1-score is 0.1521285531067762 by textrank method


### 5.2. С фильтром

In [55]:
precisions_filtered = {}
recalls_filtered = {}
f1_scores_filtered = {}

In [56]:
articles_textrank_precision_filtered, articles_textrank_recall_filtered, articles_textrank_f1_score_filtered = evaluate_keyword_extraction(golden_standard, list(filtered_textrank.values()))
precisions_filtered.update({'textrank': articles_textrank_precision_filtered})
recalls_filtered.update({'textrank': articles_textrank_recall_filtered})
f1_scores_filtered.update({'textrank': articles_textrank_f1_score_filtered})

In [57]:
articles_keybert_precision_filtered, articles_keybert_recall_filtered, articles_keybert_f1_score_filtered = evaluate_keyword_extraction(golden_standard, list(filtered_keybert.values()))
precisions_filtered.update({'keybert': articles_keybert_precision_filtered})
recalls_filtered.update({'keybert': articles_keybert_recall_filtered})
f1_scores_filtered.update({'keybert': articles_keybert_f1_score_filtered})

In [58]:
articles_gensim_precision_filtered, articles_gensim_recall_filtered, articles_gensim_f1_score_filtered = evaluate_keyword_extraction(golden_standard, list(filtered_gensim.values()))
precisions_filtered.update({'gensim': articles_gensim_precision_filtered})
recalls_filtered.update({'gensim': articles_gensim_recall_filtered})
f1_scores_filtered.update({'gensim': articles_gensim_f1_score_filtered})

In [59]:
print(f'The best average precision is {max(precisions_filtered.values())} by {list(precisions_filtered.keys())[list(precisions_filtered.values()).index(max(precisions_filtered.values()))]} method')
print(f'The best average recall is {max(recalls_filtered.values())} by {list(recalls_filtered.keys())[list(recalls_filtered.values()).index(max(recalls_filtered.values()))]} method')
print(f'The best average f1-score is {max(f1_scores_filtered.values())} by {list(f1_scores_filtered.keys())[list(f1_scores_filtered.values()).index(max(f1_scores_filtered.values()))]} method')

The best average precision is 0.08766271336229853 by textrank method
The best average recall is 0.38786583786583784 by textrank method
The best average f1-score is 0.15038174810644783 by textrank method


In [60]:
precisions_fin = {}
recalls_fin = {}
f1_scores_fin = {}

precisions_fin.update({str(list(precisions.keys())[list(precisions.values()).index(max(precisions.values()))] + ', unfiltered'): max(precisions.values())})
precisions_fin.update({str(list(precisions_filtered.keys())[list(precisions_filtered.values()).index(max(precisions_filtered.values()))] + ', filtered'): max(precisions_filtered.values())})

recalls_fin.update({str(list(recalls.keys())[list(recalls.values()).index(max(recalls.values()))] + ', unfiltered'): max(recalls.values())})
recalls_fin.update({str(list(recalls_filtered.keys())[list(recalls_filtered.values()).index(max(recalls_filtered.values()))] + ', filtered'): max(recalls_filtered.values())})

f1_scores_fin.update({str(list(f1_scores.keys())[list(f1_scores.values()).index(max(f1_scores.values()))] + ', unfiltered'): max(f1_scores.values())})
f1_scores_fin.update({str(list(f1_scores_filtered.keys())[list(f1_scores_filtered.values()).index(max(f1_scores_filtered.values()))] + ', filtered'): max(f1_scores_filtered.values())})

In [61]:
print(f'The best average precision is {max(precisions_fin.values())} by {list(precisions_fin.keys())[list(precisions_fin.values()).index(max(precisions_fin.values()))]} method')
print(f'The best average recall is {max(recalls_fin.values())} by {list(recalls_fin.keys())[list(recalls_fin.values()).index(max(recalls_fin.values()))]} method')
print(f'The best average f1-score is {max(f1_scores_fin.values())} by {list(f1_scores_fin.keys())[list(f1_scores_fin.values()).index(max(f1_scores_fin.values()))]} method')

The best average precision is 0.09661584261660632 by textrank, unfiltered method
The best average recall is 0.38786583786583784 by textrank, filtered method
The best average f1-score is 0.1521285531067762 by textrank, unfiltered method
